In [1]:
!pip install pyspark

In [2]:
import pandas as pd
import numpy as np

from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, DateType

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession 

sc = SparkContext('local') 
spark = SparkSession(sc) 

In [4]:
df = spark.read.format("com.databricks.spark.csv").options(sep=",", header=True, quote="").csv('iris.csv')

In [5]:
df

DataFrame["sepal.length: string, ""sepal.width"": string, ""petal.length"": string, ""petal.width"": string, ""variety""": string]

In [6]:
df.show(5)

+-------------+---------------+----------------+---------------+------------+
|"sepal.length|""sepal.width""|""petal.length""|""petal.width""|""variety"""|
+-------------+---------------+----------------+---------------+------------+
|         "5.1|            3.5|             1.4|             .2| ""Setosa"""|
|         "4.9|              3|             1.4|             .2| ""Setosa"""|
|         "4.7|            3.2|             1.3|             .2| ""Setosa"""|
|         "4.6|            3.1|             1.5|             .2| ""Setosa"""|
|           "5|            3.6|             1.4|             .2| ""Setosa"""|
+-------------+---------------+----------------+---------------+------------+
only showing top 5 rows



In [7]:
df.printSchema()

root
 |-- "sepal.length: string (nullable = true)
 |-- ""sepal.width"": string (nullable = true)
 |-- ""petal.length"": string (nullable = true)
 |-- ""petal.width"": string (nullable = true)
 |-- ""variety""": string (nullable = true)



In [8]:
newColumns = ["sepal_length","sepal_width","petal_length","petal_width","variety"]
df_new = df.toDF(*newColumns)

df_new = df_new.withColumn("sepal_length", regexp_replace(col("sepal_length"), '"', ""))\
    .withColumn("variety", regexp_replace(col("variety"), '"', ""))

df_new = df_new \
    .withColumn('sepal_length', col('sepal_length').cast(DoubleType())) \
    .withColumn('sepal_width', col('sepal_width').cast(DoubleType())) \
    .withColumn('petal_length', col('petal_length').cast(DoubleType())) \
    .withColumn('petal_width', col('petal_width').cast(DoubleType()))

In [9]:
df_new.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [10]:
df_new.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)



In [11]:
df_new.summary().show()

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|
|    25%|               5.1|                2.8|               1.6|               0.3|     null|
|    50%|               5.8|                3.0|               4.3|               1.3|     null|
|    75%|               6.4|                3.3|               5.1|               1.8|     null|
|    max|               7.9|  

In [13]:
pipeline = Pipeline(stages = 
[
  StringIndexer(inputCol='variety', outputCol='varietyInd'),
  VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol='Features')
]
)

In [14]:
pipelineTrained = pipeline.fit(df_new)

In [15]:
pipelineTrained.transform(df_new).show()

+------------+-----------+------------+-----------+-------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|varietyInd|         Features|
+------------+-----------+------------+-----------+-------+----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|       0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|       0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|       0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|       0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|       0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|       0.0|[5.0,3.4,1.5,0.2]|
|         4.4|       

In [16]:
df_features = pipelineTrained.transform(df_new)

In [17]:
train, test = df_features.randomSplit([0.8, 0.2], seed=12345)

In [18]:
train.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|
+------------+-----------+------------+-----------+----------+----------+-----------------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|    Setosa|       0.0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|    Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         4.6|        3.4|         1.4|        0.3|    Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         4.6|        3.6|         1.0|        0.2|    Setosa|       0.0|[4.6,3.

In [19]:
lr = LogisticRegression(featuresCol = 'Features', labelCol = 'varietyInd')
lrModel = lr.fit(train)

In [20]:
train_res = lrModel.transform(train)
test_res = lrModel.transform(test)

In [22]:
train_res.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|[67.7027916724250...|[1.0,1.6424725423...|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|[57.0087432384673...|[1.0,1.4036232732...|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|[61.7475607633488...|[1.0,2.4772189504...|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|[68.8060986698627...|[1.0,1.2455409054.

In [23]:
test_res.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|       0.0|[4.6,3.2,1.4,0.2]|[65.8143588816466...|[1.0,8.9932470888...|       0.0|
|         5.0|        3.0|         1.6|        0.2|    Setosa|       0.0|[5.0,3.0,1.6,0.2]|[52.7723413987004...|[1.0,9.3248098831...|       0.0|
|         5.0|        3.2|         1.2|        0.2|    Setosa|       0.0|[5.0,3.2,1.2,0.2]|[64.6690735917121...|[1.0,1.7992922822...|       0.0|
|         5.0|        3.5|         1.3|        0.3|    Setosa|       0.0|[5.0,3.5,1.3,0.3]|[71.4022937223270...|[1.0,5.6805141440.

In [24]:
ev = MulticlassClassificationEvaluator(labelCol='varietyInd')

In [27]:
print('Точность предсказания на тестовой выборке:', ev.evaluate(test_res)*100, '%')

Точность предсказания на тестовой выборке: 100.0 %


In [28]:
print('Точность предсказания на обучающей выборке:', ev.evaluate(train_res)*100, '%')

Точность предсказания на обучающей выборке: 98.44961240310077 %


Вывод: Анализ показал, что обученная модель на тестовой выборке показал 100 процентный результат, на обучающей выборке
точность составила 98 процентов, значит есть небольшие отклонения.